In [2]:
import torch
import numpy as np
from visual_gt_pred import *

In [14]:


num_dconv_points = 9
dcn_kernel = int(np.power(num_dconv_points, 1/2)) # 3 
dcn_pad = int((dcn_kernel - 1) / 2)
dcn_base = np.arange(-dcn_pad,  dcn_pad + 1).astype(np.float64)

dcn_base_y = np.repeat(dcn_base, dcn_kernel)
dcn_base_x = np.tile(dcn_base, dcn_kernel)
dcn_base_offset = np.stack([dcn_base_y, dcn_base_x], axis=1)
print('dcn_base_offset stack\n', dcn_base_offset)
dcn_base_offset = torch.tensor(dcn_base_offset.reshape((-1))).view(1, -1, 1, 1)


print('dcn kernel', dcn_kernel)
print('dcn pad', dcn_pad)
print('dcn base', dcn_base)

print('dcn_base_y', dcn_base_y)
print('dcn base x', dcn_base_x)
print('dcn_base_offset\n', dcn_base_offset[0,:,0,0].tolist())


dcn_base_offset stack
 [[-1. -1.]
 [-1.  0.]
 [-1.  1.]
 [ 0. -1.]
 [ 0.  0.]
 [ 0.  1.]
 [ 1. -1.]
 [ 1.  0.]
 [ 1.  1.]]
dcn kernel 3
dcn pad 1
dcn base [-1.  0.  1.]
dcn_base_y [-1. -1. -1.  0.  0.  0.  1.  1.  1.]
dcn base x [-1.  0.  1. -1.  0.  1. -1.  0.  1.]
dcn_base_offset
 [-1.0, -1.0, -1.0, 0.0, -1.0, 1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0, 1.0, -1.0, 1.0, 0.0, 1.0, 1.0]


In [7]:
def _meshgrid(x, y, z, row_major=True):
    """Generate mesh grid of x and y.

    Args:
        x (torch.Tensor): Grids of x dimension.
        y (torch.Tensor): Grids of y dimension.
        row_major (bool, optional): Whether to return y grids first.
            Defaults to True.

    Returns:
        tuple[torch.Tensor]: The mesh grids of x and y.
    """
    # use shape instead of len to keep tracing while exporting to onnx
    x2 = x[:, None]
    y2 = y[None, :]
    print('x2', x2, x2.shape)
    print('y2', y2, y2.shape)
    
    xx = x2.repeat(1, y.shape[0])
    yy = y2.repeat(x.shape[0], 1)
    
    print('xx', xx)
    print('yy', yy)
    
    xxx = x[:, None, None].repeat(1, y.shape[0], z.shape[0]) # 3dim
    yyy = y[None, :, None].repeat(x.shape[0], 1, z.shape[0])
    zzz = z[None, None, :].repeat(x.shape[0], y.shape[0], 1)
    print(xxx.shape)
    print(yyy.shape)
    print(zzz.shape)
    xxx = xxx.view(-1)
    yyy = yyy.view(-1)
    zzz = zzz.view(-1)
    if row_major:
        return xxx, yyy, zzz
    else:
        return yyy, xxx, zzz
    

x = y = z = torch.arange(3)
grids = _meshgrid(x, y, z)
grids_nx3 = torch.stack(grids, dim = 1) - 1
print(grids_nx3.shape)
grids_flatten = grids_nx3.flatten()
print(grids_flatten, grids_flatten.shape)


x2 tensor([[0],
        [1],
        [2]]) torch.Size([3, 1])
y2 tensor([[0, 1, 2]]) torch.Size([1, 3])
xx tensor([[0, 0, 0],
        [1, 1, 1],
        [2, 2, 2]])
yy tensor([[0, 1, 2],
        [0, 1, 2],
        [0, 1, 2]])
torch.Size([3, 3, 3])
torch.Size([3, 3, 3])
torch.Size([3, 3, 3])
torch.Size([27, 3])
tensor([-1, -1, -1, -1, -1,  0, -1, -1,  1, -1,  0, -1, -1,  0,  0, -1,  0,  1,
        -1,  1, -1, -1,  1,  0, -1,  1,  1,  0, -1, -1,  0, -1,  0,  0, -1,  1,
         0,  0, -1,  0,  0,  0,  0,  0,  1,  0,  1, -1,  0,  1,  0,  0,  1,  1,
         1, -1, -1,  1, -1,  0,  1, -1,  1,  1,  0, -1,  1,  0,  0,  1,  0,  1,
         1,  1, -1,  1,  1,  0,  1,  1,  1]) torch.Size([81])


In [40]:
meshgrid_torch = torch.stack(torch.meshgrid([x, y,  z]), axis = -1).reshape(-1, 3)

# aa = [print_tensor(m, g) for m, g in enumerate(meshgrid_torch)]
print(meshgrid_torch.shape)
print(meshgrid_torch - 1)



torch.Size([27, 3])
tensor([[-1, -1, -1],
        [-1, -1,  0],
        [-1, -1,  1],
        [-1,  0, -1],
        [-1,  0,  0],
        [-1,  0,  1],
        [-1,  1, -1],
        [-1,  1,  0],
        [-1,  1,  1],
        [ 0, -1, -1],
        [ 0, -1,  0],
        [ 0, -1,  1],
        [ 0,  0, -1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, -1],
        [ 0,  1,  0],
        [ 0,  1,  1],
        [ 1, -1, -1],
        [ 1, -1,  0],
        [ 1, -1,  1],
        [ 1,  0, -1],
        [ 1,  0,  0],
        [ 1,  0,  1],
        [ 1,  1, -1],
        [ 1,  1,  0],
        [ 1,  1,  1]])


In [36]:
print(meshgrid_torch[2].reshape((-1)))

tensor([0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2,
        0, 1, 2])


In [16]:
bbox_pred = torch.randn((1, 4, 3, 3))
gradient_mul = 0.1
stride = 1
print(bbox_pred, bbox_pred.shape)

tensor([[[[-0.0782,  1.3287,  0.0268],
          [ 0.9699, -1.4672,  0.1441],
          [-0.3651, -0.7981,  0.7844]],

         [[-0.3078,  1.1395, -0.8255],
          [ 0.4193,  0.0405,  0.3509],
          [ 0.9451, -1.1516, -0.6292]],

         [[ 0.2406, -0.6891, -0.0252],
          [ 2.9733, -0.3231, -0.4262],
          [ 1.1185,  0.6263,  1.7979]],

         [[ 0.0432,  0.2825, -1.0464],
          [ 0.7600,  1.3766, -0.1705],
          [-0.0144,  0.0650, -0.1380]]]]) torch.Size([1, 4, 3, 3])



self.dcn_base_offset: 9x2, 
[[-1. -1.]
[-1.  0.]
[-1.  1.]
[ 0. -1.]
[ 0.  0.]
[ 0.  1.]
[ 1. -1.]
[ 1.  0.]
[ 1.  1.]]

[-1.0, -1.0, -1.0, 0.0, -1.0, 1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0, 1.0, -1.0, 1.0, 0.0, 1.0, 1.0]


An offset is like `[y0, x0, y1, x1, y2, x2, ..., y8, x8]`.
The spatial arrangement is like:
.. code:: text
    (x0, y0) (x1, y1) (x2, y2)
    (x3, y3) (x4, y4) (x5, y5)
    (x6, y6) (x7, y7) (x8, y8)


In [17]:

bbox_pred = torch.randn((1, 4, 3, 3))
dcn_base_offset = dcn_base_offset.type_as(bbox_pred) # (1, 18, 1, 1), 9 points
bbox_pred_grad_mul = (1 - gradient_mul) * bbox_pred.detach() + gradient_mul * bbox_pred
# map to the feature map scale
bbox_pred_grad_mul = bbox_pred_grad_mul / stride
N, C, H, W = bbox_pred.size()

l_x1 = bbox_pred_grad_mul[:, 0, :, :] # l
t_y1 = bbox_pred_grad_mul[:, 1, :, :] # t
r_x2 = bbox_pred_grad_mul[:, 2, :, :] # r
b_y2 = bbox_pred_grad_mul[:, 3, :, :] # b
bbox_pred_grad_mul_offset = bbox_pred.new_zeros( N, 2 * num_dconv_points, H, W)

bbox_pred_grad_mul_offset[:, 0, :, :] = -1.0 * t_y1  # -y1, t
bbox_pred_grad_mul_offset[:, 1, :, :] = -1.0 * l_x1  # -x1, l
bbox_pred_grad_mul_offset[:, 2, :, :] = -1.0 * t_y1  # -y1, t
bbox_pred_grad_mul_offset[:, 4, :, :] = -1.0 * t_y1  # -y1, t
bbox_pred_grad_mul_offset[:, 5, :, :] = r_x2  # x2
bbox_pred_grad_mul_offset[:, 7, :, :] = -1.0 * l_x1  # -x1
bbox_pred_grad_mul_offset[:, 11, :, :] = r_x2  # x2
bbox_pred_grad_mul_offset[:, 12, :, :] = b_y2  # y2
bbox_pred_grad_mul_offset[:, 13, :, :] = -1.0 * l_x1  # -x1
bbox_pred_grad_mul_offset[:, 14, :, :] = b_y2  # y2
bbox_pred_grad_mul_offset[:, 16, :, :] = b_y2  # y2
bbox_pred_grad_mul_offset[:, 17, :, :] = r_x2  # x2
# dcn_offset = bbox_pred_grad_mul_offset - dcn_base_offset
print_tensor('official', bbox_pred_grad_mul_offset)

official <class 'torch.Tensor'> torch.Size([1, 18, 3, 3]) torch.float32 tensor(-1.8293) tensor(1.4824)


In [18]:
bbox_pred_grad_mul_offset_new = bbox_pred.new_zeros( N, 2 * num_dconv_points, H, W)
points_nx2 = dcn_base_offset[0, :, 0, 0].reshape(-1, 2)
print_tensor('p nx2', points_nx2)
for i, point2 in enumerate(points_nx2):
    y_sign, x_sign = point2 # (-1, -1)
    y_ix, x_ix = i * 2, (i * 2 + 1)
    if y_sign < 0:  y_offset = y_sign * t_y1 
    elif y_sign > 0: y_offset = y_sign * b_y2
    else: y_offset = None

    if x_sign < 0: x_offset= x_sign * l_x1
    elif x_sign > 0: x_offset = x_sign * r_x2
    else: x_offset = None
    
    if y_offset is not None:
        bbox_pred_grad_mul_offset_new[:, y_ix, ...] = y_offset
    if x_offset is not None:
        bbox_pred_grad_mul_offset_new[:, x_ix, ...] = x_offset
    
print_tensor('official', bbox_pred_grad_mul_offset)
print_tensor('mine', bbox_pred_grad_mul_offset_new)
compare_mat = bbox_pred_grad_mul_offset_new - bbox_pred_grad_mul_offset
print(compare_mat.sum())


p nx2 <class 'torch.Tensor'> torch.Size([9, 2]) torch.float32 tensor(-1.) tensor(1.)
official <class 'torch.Tensor'> torch.Size([1, 18, 3, 3]) torch.float32 tensor(-1.8293) tensor(1.4824)
mine <class 'torch.Tensor'> torch.Size([1, 18, 3, 3]) torch.float32 tensor(-1.8293) tensor(1.4824)
tensor(0.)


In [21]:
print(l_x1)
print(bbox_pred_grad_mul_offset[:, 1, ...])
print(bbox_pred_grad_mul_offset_new[:, 1, ...])

tensor([[[ 0.1603, -0.5403,  0.8983],
         [-0.1375,  0.6144, -0.3559],
         [ 0.1795,  1.8293,  0.6029]]])
tensor([[[-0.1603,  0.5403, -0.8983],
         [ 0.1375, -0.6144,  0.3559],
         [-0.1795, -1.8293, -0.6029]]])
tensor([[[-0.1603,  0.5403, -0.8983],
         [ 0.1375, -0.6144,  0.3559],
         [-0.1795, -1.8293, -0.6029]]])
